In [1]:
# !pip install datasets transformers==4.18.0 sentencepiece

In [2]:
from datasets import *
from transformers import *
from tokenizers import *
import os
import json

In [3]:
# https://wikidocs.net/166816
# download and prepare cc_news dataset
#dataset = load_dataset("cc_news", split="train")
dataset = load_dataset("csv", data_files='issue_data.csv', delimiter=',', column_names=['text'], split="train")

Found cached dataset csv (C:/Users/qja19/.cache/huggingface/datasets/csv/default-a327c0e84e2492f5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


In [4]:
# split the dataset into training (90%) and testing (10%)
d = dataset.train_test_split(test_size=0.1)
d["train"], d["test"]

(Dataset({
     features: ['text'],
     num_rows: 39535
 }),
 Dataset({
     features: ['text'],
     num_rows: 4393
 }))

In [5]:
for t in d["train"]["text"][:3]:
  print(t)
  print("="*50)

🐛 include setext heading levels (Markdown)
terrible ui decision in the source control dropdown
now that I have cooled off from rage: let's put all the most used elements of the menu in the top of it.

- push
- pull 
- commit
totally agree
/duplicate of #103923
Thanks for creating this issue! We figured it's covering the same as another one we already have. Thus, we closed this one as a duplicate. You can search for existing issues [here](https://aka.ms/vscodeissuesearch). See also our [issue reporting](https://aka.ms/vscodeissuereporting) guidelines.

Happy Coding!
As a workaround I just published a [simple extension](https://marketplace.visualstudio.com/items?itemName=georgejames.gitfavorites).
Add a toolbar with some basic actions


In [6]:
# if you have your custom dataset 
# dataset = LineByLineTextDataset(
#     tokenizer=tokenizer,
#     file_path="path/to/data.txt",
#     block_size=64,
# )

In [7]:
# or if you have huge custom dataset separated into files
# load the splitted files
# files = ["train1.txt", "train2.txt"] # train3.txt, etc.
# dataset = load_dataset("text", data_files=files, split="train")

In [8]:
# if you want to train the tokenizer from scratch (especially if you have custom
# dataset loaded as datasets object), then run this cell to save it as files
# but if you already have your custom data as text files, there is no point using this
def dataset_to_text(dataset, output_filename="data.txt"):
  """Utility function to save dataset text to disk,
  useful for using the texts to train the tokenizer 
  (as the tokenizer accepts files)"""
  with open(output_filename, "w") as f:
    for t in dataset["text"]:
      print(t, file=f)

# save the training set to train.txt
dataset_to_text(d["train"], "train.txt")
# save the testing set to test.txt
dataset_to_text(d["test"], "test.txt")

In [9]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]
# if you want to train the tokenizer on both sets
# files = ["train.txt", "test.txt"]
# training the tokenizer on the training set
files = ["train.txt"]
# 30,522 vocab is BERT's default vocab size, feel free to tweak
vocab_size = 30_522
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 512
# whether to truncate
truncate_longer_samples = False

In [10]:
# initialize the WordPiece tokenizer
tokenizer = BertWordPieceTokenizer()
# train the tokenizer
tokenizer.train(files=files, vocab_size=vocab_size, special_tokens=special_tokens)
# enable truncation up to the maximum 512 tokens
tokenizer.enable_truncation(max_length=max_length)

In [11]:
model_path = "pretrained-bert"
# make the directory if not already there
if not os.path.isdir(model_path):
  os.mkdir(model_path)

In [12]:
# save the tokenizer  
tokenizer.save_model(model_path)

['pretrained-bert\\vocab.txt']

In [13]:
# dumping some of the tokenizer config to config file, 
# including special tokens, whether to lower case and the maximum sequence length
with open(os.path.join(model_path, "config.json"), "w") as f:
  tokenizer_cfg = {
      "do_lower_case": True,
      "unk_token": "[UNK]",
      "sep_token": "[SEP]",
      "pad_token": "[PAD]",
      "cls_token": "[CLS]",
      "mask_token": "[MASK]",
      "model_max_length": max_length,
      "max_len": max_length,
  }
  json.dump(tokenizer_cfg, f)

In [14]:
# when the tokenizer is trained and configured, load it as BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(model_path)

Didn't find file pretrained-bert\tokenizer.json. We won't load it.
Didn't find file pretrained-bert\added_tokens.json. We won't load it.
Didn't find file pretrained-bert\special_tokens_map.json. We won't load it.
Didn't find file pretrained-bert\tokenizer_config.json. We won't load it.
loading file pretrained-bert\vocab.txt
loading file None
loading file None
loading file None
loading file None
loading configuration file pretrained-bert\config.json
Model config BertConfig {
  "_name_or_path": "pretrained-bert",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "cls_token": "[CLS]",
  "do_lower_case": true,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "mask_token": "[MASK]",
  "max_len": 512,
  "max_position_embeddings": 512,
  "model_max_length": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token": "[PA

In [15]:
def encode_with_truncation(examples):
  """Mapping function to tokenize the sentences passed with truncation"""
  return tokenizer(examples["text"], truncation=True, padding="max_length",
                   max_length=max_length, return_special_tokens_mask=True)

def encode_without_truncation(examples):
  """Mapping function to tokenize the sentences passed without truncation"""
  return tokenizer(examples["text"], return_special_tokens_mask=True)

# the encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation

# tokenizing the train dataset
train_dataset = d["train"].map(encode, batched=True)
# tokenizing the testing dataset
test_dataset = d["test"].map(encode, batched=True)

if truncate_longer_samples:
  # remove other columns and set input_ids and attention_mask as PyTorch tensors
  train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
  test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
  # remove other columns, and remain them as Python lists
  test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
  train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])

Map:   0%|          | 0/39535 [00:00<?, ? examples/s]

Map:   0%|          | 0/4393 [00:00<?, ? examples/s]

In [16]:
from itertools import chain
# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
# grabbed from: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a
# remainder for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value
# might be slower to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map
if not truncate_longer_samples:
  train_dataset = train_dataset.map(group_texts, batched=True,
                                    desc=f"Grouping texts in chunks of {max_length}")
  test_dataset = test_dataset.map(group_texts, batched=True,
                                  desc=f"Grouping texts in chunks of {max_length}")
  # convert them from lists to torch tensors
  train_dataset.set_format("torch")
  test_dataset.set_format("torch")

Grouping texts in chunks of 512:   0%|          | 0/39535 [00:00<?, ? examples/s]

Grouping texts in chunks of 512:   0%|          | 0/4393 [00:00<?, ? examples/s]

In [17]:
for i in train_dataset:
    print(i['input_ids'].dtype,
        i['token_type_ids'].dtype,
        i['attention_mask'].dtype)
    break

torch.int64 torch.int64 torch.int64


In [18]:
len(train_dataset), len(test_dataset)

(169928, 9565)

In [19]:
# initialize the model with the config
model_config = BertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)
model = BertForMaskedLM(config=model_config)

In [20]:
# initialize the data collator, randomly masking 20% (default is 15%) of the tokens for the Masked Language
# Modeling (MLM, Masked Language Model) task
# 텍스트의 각 배치(batch)에서 일부 토큰을 무작위로 마스킹
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
)

In [22]:
training_args = TrainingArguments(
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=10,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=16, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,  # evaluation batch size
    logging_steps=1000,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=1000,
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

#training_args = training_args.set_dataloader(train_batch_size=64, eval_batch_size=64)

using `logging_steps` to initialize `eval_steps` to 1000
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
# initialize the trainer and pass everything to it
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [24]:
# train the model
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask, text. If special_tokens_mask, text are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
c:\Users\qja19\Anaconda3\envs\bert\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 169928
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 13270


  0%|          | 0/13270 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# when you load from pretrained
#model = BertForMaskedLM.from_pretrained(os.path.join(model_path, "checkpoint-6000"))
#tokenizer = BertTokenizerFast.from_pretrained(model_path)
# or simply use pipeline
fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer, device=0)

In [ ]:
# perform predictions
# https://huggingface.co/docs/transformers/main_classes/pipelines
#example = "It is known that [MASK] is the capital of Germany"
# jsonl
example = """
I have 3 input datasets and 2 output sets. My input: 1.[MASK] file, 2.yaml file, 3. gzip library as a dataset.
"""
for prediction in fill_mask(example):
  print(prediction)

{'score': 0.08583386242389679, 'token': 20, 'token_str': '.', 'sequence': 'i have 3 input datasets and 2 output sets. my input : 1.. file, 2. yaml file, 3. gzip library as a dataset.'}
{'score': 0.03333842381834984, 'token': 18, 'token_str': ',', 'sequence': 'i have 3 input datasets and 2 output sets. my input : 1., file, 2. yaml file, 3. gzip library as a dataset.'}
{'score': 0.021261252462863922, 'token': 870, 'token_str': 'you', 'sequence': 'i have 3 input datasets and 2 output sets. my input : 1. you file, 2. yaml file, 3. gzip library as a dataset.'}
{'score': 0.019690483808517456, 'token': 822, 'token_str': 'the', 'sequence': 'i have 3 input datasets and 2 output sets. my input : 1. the file, 2. yaml file, 3. gzip library as a dataset.'}
{'score': 0.01707284525036812, 'token': 827, 'token_str': 'is', 'sequence': 'i have 3 input datasets and 2 output sets. my input : 1. is file, 2. yaml file, 3. gzip library as a dataset.'}


In [ ]:
# perform predictions
examples = [
  "Today's most trending hashtags on [MASK] is Donald Trump",
  "The [MASK] was cloudy yesterday, but today it's rainy.",
]
for example in examples:
  for prediction in fill_mask(example):
    print(f"{prediction['sequence']}, confidence: {prediction['score']}")
  print("="*50)

today's most trending hashtags on the is donald trump, confidence: 0.021932044997811317
today's most trending hashtags on. is donald trump, confidence: 0.020079581066966057
today's most trending hashtags on to is donald trump, confidence: 0.01751154102385044
today's most trending hashtags on in is donald trump, confidence: 0.014734507538378239
today's most trending hashtags on, is donald trump, confidence: 0.014009387232363224
the : was cloudy yesterday, but today it's rainy., confidence: 0.04903589189052582
the'was cloudy yesterday, but today it's rainy., confidence: 0.0413290411233902
the to was cloudy yesterday, but today it's rainy., confidence: 0.015920132398605347
the not was cloudy yesterday, but today it's rainy., confidence: 0.015270046889781952
the code was cloudy yesterday, but today it's rainy., confidence: 0.01079642865806818
